In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_predict
from numpy import mean
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Margin_new.xlsx to Margin_new.xlsx


In [4]:
df_original = pd.read_excel('Margin_new.xlsx')

In [5]:
df_original

,CustomerID,CampaignID,PaymentType,VehicleTypeID,BranchID,GenderCode,new_OccupationName,new_age,Re-buy,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent
0,2,189,Finance,1,12,100000000,เจ้าของธุรกิจ/Business Owner,56,0,8.0,30.0,60,0.0
1,3,43,Finance,1,30,100000000,เจ้าของธุรกิจ/Business Owner,56,0,5.0,33.0,36,2.0
2,8,72,Finance,1,2,100000000,เจ้าของธุรกิจ/Business Owner,56,0,2.0,25.0,60,0.0
3,18,750,Finance,12,2,100000000,เจ้าของธุรกิจ/Business Owner,56,0,8.0,25.0,60,3.0
4,19,39,Finance,1,9,100000000,เจ้าของธุรกิจ/Business Owner,56,0,8.0,25.0,60,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6561,19346,22,Cash,1,2,100000001,แม่บ้าน/Housewife,57,0,7.0,0.0,0,0.0
6562,19348,34,Finance,2,3,100000000,ผู้บริหารระดับสูง/Top Management,50,0,19.0,0.0,60,3.0
6563,19349,759,Cash,1,11,100000000,แพทย์/พยาบาล/ทันตแพทย์/Doctor/Nurse/Dentist,63,0,3.0,0.0,0,0.0
6564,19355,796,Finance,1,2,100000000,ผู้บริหารระดับสูง/Top Management,49,0,2.0,20.0,60,6.0


In [6]:
df_groupby = df_original.copy()

In [7]:
cols = [1,2,3,4,5,6,7,8]
df_groupby = df_groupby.drop(df_groupby.columns[cols],axis=1)


In [8]:
df_groupby = df_groupby.groupby(['CustomerID']).mean()
df_groupby

,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent
CustomerID,,,,
2,8.0,30.0,60.0,0.0
3,5.0,33.0,36.0,2.0
8,2.0,25.0,60.0,0.0
18,8.0,25.0,60.0,3.0
19,8.0,25.0,60.0,0.0
...,...,...,...,...
19346,7.0,0.0,0.0,0.0
19348,19.0,0.0,60.0,3.0
19349,3.0,0.0,0.0,0.0


In [9]:
df_catagories = df_original.copy()


In [10]:
cols = [1,2,5,7,6,8,9,10,11,12]
df_catagories = df_catagories.drop(df_catagories.columns[cols],axis=1)


In [11]:
df_catagories = (df_catagories.set_index('CustomerID').groupby(level='CustomerID').apply(lambda g: g.apply(pd.value_counts)).unstack(level=1).fillna(0))

In [12]:
df_catagories


VehicleTypeID                           ... BranchID                         
                      1    2    3    6    7    8   ...       10   11   12   30   45   63
CustomerID                                         ...                                  
2                    1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  1.0  0.0  0.0  0.0
3                    1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  1.0  0.0  0.0
8                    1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
18                   0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
19                   1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
...                  ...  ...  ...  ...  ...  ...  ...      ...  ...  ...  ...  ...  ...
19346                1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
19348                0.0  1.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
19349                1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  1.0  0.0  0.0  0.0  0.0
19355                1.0  0.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0
19369                0.0  1.0  0.0  0.0  0.0  0.0  ...      0.0  0.0  0.0  0.0  0.0  0.0

[6227 rows x 26 columns]

In [13]:
df_main = df_original.copy()

In [14]:
df_main = df_main.drop(df_main.columns.difference(['CustomerID','Re-buy']), axis = 1)
df_main = df_main.drop_duplicates(subset=['CustomerID'])

In [15]:
first_merged_inner = df_groupby.merge(df_catagories,left_on='CustomerID',right_on='CustomerID')
df_final = df_main.merge(first_merged_inner,left_on='CustomerID',right_on='CustomerID')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [16]:
df_final

,CustomerID,Re-buy,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent,"(VehicleTypeID, 1)","(VehicleTypeID, 2)","(VehicleTypeID, 3)","(VehicleTypeID, 6)","(VehicleTypeID, 7)","(VehicleTypeID, 8)","(VehicleTypeID, 9)","(VehicleTypeID, 10)","(VehicleTypeID, 11)","(VehicleTypeID, 12)","(VehicleTypeID, 30)","(VehicleTypeID, 45)","(VehicleTypeID, 63)","(BranchID, 1)","(BranchID, 2)","(BranchID, 3)","(BranchID, 6)","(BranchID, 7)","(BranchID, 8)","(BranchID, 9)","(BranchID, 10)","(BranchID, 11)","(BranchID, 12)","(BranchID, 30)","(BranchID, 45)","(BranchID, 63)"
0,2,0,8.0,30.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,0,5.0,33.0,36.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,8,0,2.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,18,0,8.0,25.0,60.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19,0,8.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,19346,0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6223,19348,0,19.0,0.0,60.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6224,19349,0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6225,19355,0,2.0,20.0,60.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_svm1 = df_final.copy()
df_svm1 = df_svm1.dropna()
df_svm1 = df_svm1.drop(columns=['CustomerID'],axis=1)

In [18]:
df_svm1

,Re-buy,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent,"(VehicleTypeID, 1)","(VehicleTypeID, 2)","(VehicleTypeID, 3)","(VehicleTypeID, 6)","(VehicleTypeID, 7)","(VehicleTypeID, 8)","(VehicleTypeID, 9)","(VehicleTypeID, 10)","(VehicleTypeID, 11)","(VehicleTypeID, 12)","(VehicleTypeID, 30)","(VehicleTypeID, 45)","(VehicleTypeID, 63)","(BranchID, 1)","(BranchID, 2)","(BranchID, 3)","(BranchID, 6)","(BranchID, 7)","(BranchID, 8)","(BranchID, 9)","(BranchID, 10)","(BranchID, 11)","(BranchID, 12)","(BranchID, 30)","(BranchID, 45)","(BranchID, 63)"
0,0,8.0,30.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,5.0,33.0,36.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,2.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,8.0,25.0,60.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,8.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6223,0,19.0,0.0,60.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6224,0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6225,0,2.0,20.0,60.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X = df_svm1.drop('Re-buy',axis=1)
y = df_svm1['Re-buy']

In [20]:
X

,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent,"(VehicleTypeID, 1)","(VehicleTypeID, 2)","(VehicleTypeID, 3)","(VehicleTypeID, 6)","(VehicleTypeID, 7)","(VehicleTypeID, 8)","(VehicleTypeID, 9)","(VehicleTypeID, 10)","(VehicleTypeID, 11)","(VehicleTypeID, 12)","(VehicleTypeID, 30)","(VehicleTypeID, 45)","(VehicleTypeID, 63)","(BranchID, 1)","(BranchID, 2)","(BranchID, 3)","(BranchID, 6)","(BranchID, 7)","(BranchID, 8)","(BranchID, 9)","(BranchID, 10)","(BranchID, 11)","(BranchID, 12)","(BranchID, 30)","(BranchID, 45)","(BranchID, 63)"
0,8.0,30.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,5.0,33.0,36.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.0,25.0,60.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,25.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6223,19.0,0.0,60.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6224,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6225,2.0,20.0,60.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.preprocessing import StandardScaler
X_scaled = X.copy()

col_names = ['AvgMargin', 'AvgDownPercent','AvgTerm','AvgInterestPercent']
features = X_scaled[col_names]

# Use scaler of choice; here Standard scaler is used
scaler = StandardScaler().fit(features.values)
features = scaler.fit_transform(features.values)

X_scaled[col_names] = features

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25,random_state = 1000)

In [23]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear',random_state=100,probability=True)
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=100, shrinking=True, tol=0.001,
    verbose=False)

In [24]:
def run_kernel(X_train_scaled: pd.DataFrame , y_train: pd.DataFrame, X_test_scaled: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame: 
    name=[]
    models = [
          ('Linear', SVC(kernel='linear')), 
          ('RBF', SVC(kernel='rbf')),
            ]
    target_names = ['No Re-buy', 'Re-buy']
    for name, model in models:
        clf = model.fit(X_train, y_train)
        y_test_pred = clf.predict(X_test)
        y_train_pred = clf.predict(X_train)
        print(name)
        print(classification_report(y_train, y_train_pred, target_names=target_names))
        print(classification_report(y_test, y_test_pred, target_names=target_names))
      



In [25]:
run_kernel(X_train,y_train,X_test,y_test)

Linear
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      1.00      4407
      Re-buy       1.00      0.87      0.93       263

    accuracy                           0.99      4670
   macro avg       1.00      0.93      0.96      4670
weighted avg       0.99      0.99      0.99      4670

              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      0.99      1459
      Re-buy       1.00      0.82      0.90        98

    accuracy                           0.99      1557
   macro avg       0.99      0.91      0.95      1557
weighted avg       0.99      0.99      0.99      1557

RBF
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      1.00      4407
      Re-buy       1.00      0.87      0.93       263

    accuracy                           0.99      4670
   macro avg       1.00      0.93      0.96      4670
weighted avg       0.99      0.99      0.99      4670

        

In [26]:
y_pred = svclassifier.predict(X_test)
y_train_pred = svclassifier.predict(X_train)

In [27]:
print(pd.crosstab(y_train, y_train_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_train, y_train_pred))

print(pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y_test,y_pred))

Predicted     0    1   All
Actual                    
0          4407    0  4407
1            35  228   263
All        4442  228  4670
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4407
           1       1.00      0.87      0.93       263

    accuracy                           0.99      4670
   macro avg       1.00      0.93      0.96      4670
weighted avg       0.99      0.99      0.99      4670

Predicted     0   1   All
Actual                   
0          1459   0  1459
1            18  80    98
All        1477  80  1557
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1459
           1       1.00      0.82      0.90        98

    accuracy                           0.99      1557
   macro avg       0.99      0.91      0.95      1557
weighted avg       0.99      0.99      0.99      1557



In [28]:
y_pred_proba_all = svclassifier.predict_proba(X_scaled)
y_pred_all  = svclassifier.predict(X_scaled)

In [29]:
print(pd.crosstab(y, y_pred_all, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(y,y_pred_all))

Predicted     0    1   All
Actual                    
0          5866    0  5866
1            53  308   361
All        5919  308  6227
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5866
           1       1.00      0.85      0.92       361

    accuracy                           0.99      6227
   macro avg       1.00      0.93      0.96      6227
weighted avg       0.99      0.99      0.99      6227



In [30]:
df_result = df_final.copy()

In [31]:
df_result = df_result.drop(df_result.columns.difference(['CustomerID','Re-buy','new_age']), axis = 1)

In [32]:
df_result['Re-buy Predict'] = y_pred_all
df_result['Re-buy Proba'] = y_pred_proba_all[:,1:]
df_result

,CustomerID,Re-buy,Re-buy Predict,Re-buy Proba
0,2,0,0,0.008435
1,3,0,0,0.008434
2,8,0,0,0.008440
3,18,0,0,0.008438
4,19,0,0,0.008434
...,...,...,...,...
6222,19346,0,0,0.008439
6223,19348,0,0,0.008434
6224,19349,0,0,0.008435
6225,19355,0,0,0.008440


In [33]:
result = pd.DataFrame(X.columns.values,columns=['Attribute'])
coef = svclassifier.coef_
coef = np.transpose(coef)
result['coef']  = coef
result.sort_values(by=['coef'],ascending=False)

,Attribute,coef
4,"(VehicleTypeID, 1)",1.400152e+00
13,"(VehicleTypeID, 12)",1.400145e+00
5,"(VehicleTypeID, 2)",1.400115e+00
8,"(VehicleTypeID, 7)",1.399764e+00
7,"(VehicleTypeID, 6)",1.000000e+00
29,"(BranchID, 63)",6.002255e-01
22,"(BranchID, 8)",6.001402e-01
18,"(BranchID, 2)",6.000931e-01
21,"(BranchID, 7)",6.000762e-01
19,"(BranchID, 3)",6.000652e-01


In [34]:
from google.colab import files
result.to_csv('result.csv') 
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
def run_exps(X_train_scaled: pd.DataFrame , y_train: pd.DataFrame, X_test_scaled: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame: 
    dfs = []
    models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()), 
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
        ]
    results = []
    names = []
    scoring = ['accuracy', 'precision', 'recall', 'f1_weighted', 'roc_auc']
    target_names = ['No Re-buy', 'Re-buy']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=3, shuffle=True, random_state=100)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        print(pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        final = pd.concat(dfs, ignore_index=True)
    return final

In [36]:
run_exps(X_train,y_train,X_test,y_test)

LogReg
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      0.99      1459
      Re-buy       1.00      0.81      0.89        98

    accuracy                           0.99      1557
   macro avg       0.99      0.90      0.94      1557
weighted avg       0.99      0.99      0.99      1557

Predicted     0   1   All
Actual                   
0          1459   0  1459
1            19  79    98
All        1478  79  1557
RF
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      0.99      1459
      Re-buy       1.00      0.79      0.88        98

    accuracy                           0.99      1557
   macro avg       0.99      0.89      0.94      1557
weighted avg       0.99      0.99      0.99      1557

Predicted     0   1   All
Actual                   
0          1459   0  1459
1            21  77    98
All        1480  77  1557
KNN
              precision    recall  f1-score   support

   No Re-buy     

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_weighted,test_roc_auc,model
0,0.042594,0.011501,0.991651,0.987500,0.868132,0.991398,0.949729,LogReg
1,0.031051,0.010952,0.993577,1.000000,0.866667,0.993359,0.950715,LogReg
2,0.040447,0.010945,0.989075,1.000000,0.824742,0.988582,0.914526,LogReg
3,0.321466,0.066095,0.987797,0.986486,0.802198,0.987204,0.936041,RF
4,0.319127,0.063042,0.991651,1.000000,0.826667,0.991273,0.946928,RF
5,0.288560,0.059742,0.985861,0.987013,0.783505,0.985097,0.924899,RF
6,0.015064,0.279504,0.974310,1.000000,0.560440,0.970864,0.879350,KNN
7,0.011224,0.202716,0.978805,1.000000,0.560000,0.975933,0.851781,KNN
8,0.011205,0.179257,0.969794,1.000000,0.515464,0.965205,0.844375,KNN
9,0.092660,0.051098,0.990366,0.963415,0.868132,0.990130,0.940224,SVM


In [37]:
from google.colab import files
uploaded = files.upload()


Saving all_data.xlsx to all_data.xlsx


In [38]:
All_data = pd.read_excel('all_data.xlsx')

In [39]:
All_data

,CustomerID,Re-buy,VehicleTypeID,BranchID,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent
0,1,0,2,9,0.0,0.0,0.0,0.0
1,2,0,1,7,0.0,0.0,0.0,0.0
2,3,0,1,9,0.0,0.0,0.0,0.0
3,4,0,1,11,0.0,0.0,0.0,0.0
4,5,0,1,9,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
11600,11467,0,1,11,9.0,25.0,60.0,6.0
11601,11468,0,1,8,8.0,25.0,60.0,6.0
11602,11469,0,1,3,22.0,25.0,60.0,2.0
11603,11470,0,1,9,6.0,25.0,60.0,7.0


In [40]:
All_Customer = All_data.copy()

In [41]:
All_Customer = All_Customer.drop(All_Customer.columns.difference(['CustomerID','Re-buy']), axis = 1)
All_Customer = All_Customer.drop_duplicates(subset=['CustomerID'])


In [42]:
All_Customer_para = All_data.copy()

In [43]:
cols = [1,2,3]
All_Customer_para = All_Customer_para.drop(All_Customer_para.columns[cols],axis=1)

In [44]:
All_Customer_para = All_Customer_para.groupby(['CustomerID']).mean()

In [45]:
All_Customer_Attribute = All_data.copy()

In [46]:
cols = [1,4,5,6,7]
All_Customer_Attribute = All_Customer_Attribute.drop(All_Customer_Attribute.columns[cols],axis=1)

In [47]:
All_Customer_Attribute = (All_Customer_Attribute.set_index('CustomerID').groupby(level='CustomerID').apply(lambda g: g.apply(pd.value_counts)).unstack(level=1).fillna(0))

In [48]:
Customer_merged_inner = All_Customer_para.merge(All_Customer_Attribute,left_on='CustomerID',right_on='CustomerID')
Customer_Final = All_Customer.merge(Customer_merged_inner,left_on='CustomerID',right_on='CustomerID')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [49]:
Customer_Final

,CustomerID,Re-buy,AvgMargin,AvgDownPercent,AvgTerm,AvgInterestPercent,"(VehicleTypeID, 1)","(VehicleTypeID, 2)","(VehicleTypeID, 3)","(VehicleTypeID, 6)","(VehicleTypeID, 7)","(VehicleTypeID, 8)","(VehicleTypeID, 9)","(VehicleTypeID, 10)","(VehicleTypeID, 11)","(VehicleTypeID, 12)","(VehicleTypeID, 30)","(VehicleTypeID, 45)","(VehicleTypeID, 63)","(BranchID, 1)","(BranchID, 2)","(BranchID, 3)","(BranchID, 6)","(BranchID, 7)","(BranchID, 8)","(BranchID, 9)","(BranchID, 10)","(BranchID, 11)","(BranchID, 12)","(BranchID, 30)","(BranchID, 45)","(BranchID, 63)"
0,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11466,11467,0,9.0,25.0,60.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11467,11468,0,8.0,25.0,60.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11468,11469,0,22.0,25.0,60.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11469,11470,0,6.0,25.0,60.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
SVM2 = Customer_Final.copy()

In [51]:
All_X = SVM2.drop(['CustomerID','Re-buy'],axis=1)
All_y = SVM2['Re-buy']

In [52]:
All_X_scaled = All_X.copy()

col_names = ['AvgMargin', 'AvgDownPercent','AvgTerm','AvgInterestPercent']
features = All_X_scaled[col_names]

# Use scaler of choice; here Standard scaler is used
scaler = StandardScaler().fit(features.values)
features = scaler.fit_transform(features.values)

All_X_scaled[col_names] = features

In [53]:
from sklearn.model_selection import train_test_split
All_X_train, All_X_test, All_y_train, All_y_test = train_test_split(All_X_scaled, All_y, test_size = 0.4,random_state = 1234567)

In [54]:
Customer_y_pred_proba = svclassifier.predict_proba(All_X_train)
Customer_y_pred_all  = svclassifier.predict(All_X_train)

In [55]:
print(pd.crosstab(All_y_train, Customer_y_pred_all, rownames = ['Actual'], colnames =['Predicted'], margins = True))
print(classification_report(All_y_train,Customer_y_pred_all))

Predicted     0   1   All
Actual                   
0          6808   0  6808
1             0  74    74
All        6808  74  6882
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6808
           1       1.00      1.00      1.00        74

    accuracy                           1.00      6882
   macro avg       1.00      1.00      1.00      6882
weighted avg       1.00      1.00      1.00      6882



In [56]:
Customer_result = All_Customer.copy()

In [ ]:
Customer_result['Re-buy Predict'] = Customer_y_pred_all
Customer_result['Re-buy Proba'] = Customer_y_pred_proba[:,1:]
Customer_result.sort_values(['Re-buy Proba'],ascending=False)

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
def run_predict(X_train_scaled: pd.DataFrame , y_train: pd.DataFrame, X_test_scaled: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame: 
    dfs = []
    models = [
          ('LogReg', LogisticRegression()), 
          ('SVM', SVC()), 
          ('XGB', XGBClassifier())
        ]
    results = []
    names = []
    scoring = ['accuracy', 'precision', 'recall', 'f1_weighted', 'roc_auc']
    target_names = ['No Re-buy', 'Re-buy']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True,random_state=999)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(All_X_train,All_y_train)
        cv_y_pred = model_selection.cross_val_predict(model,X_scaled,y,cv=kfold)
        y_pred = clf.predict(All_X_test)
        y_pred_to_margin = clf.predict(X_scaled)
        print(name)
        print('y_pred')
        print(classification_report(All_y_test, y_pred, target_names=target_names))
        print('y_pred_to_margin')
        print(classification_report(y, y_pred_to_margin, target_names=target_names))
        print('cv_y_pred')
        print(classification_report(y, cv_y_pred, target_names=target_names))
        print('y_pred')
        print(pd.crosstab(All_y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
        print('y_pred_to_margin')
        print(pd.crosstab(y, y_pred_to_margin, rownames = ['Actual'], colnames =['Predicted'], margins = True))
        print('cv_y_pred')
        print(pd.crosstab(y, cv_y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True))
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        final = pd.concat(dfs, ignore_index=True)
    return final

In [65]:
run_predict(X_train,y_train,X_test,y_test)

LogReg
y_pred
              precision    recall  f1-score   support

   No Re-buy       1.00      1.00      1.00      4538
      Re-buy       1.00      0.92      0.96        51

    accuracy                           1.00      4589
   macro avg       1.00      0.96      0.98      4589
weighted avg       1.00      1.00      1.00      4589

y_pred_to_margin
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      0.99      5866
      Re-buy       1.00      0.76      0.86       361

    accuracy                           0.99      6227
   macro avg       0.99      0.88      0.93      6227
weighted avg       0.99      0.99      0.98      6227

cv_y_pred
              precision    recall  f1-score   support

   No Re-buy       0.99      1.00      1.00      5866
      Re-buy       1.00      0.84      0.91       361

    accuracy                           0.99      6227
   macro avg       1.00      0.92      0.95      6227
weighted avg       0.99      0.99

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1_weighted,test_roc_auc,model
0,0.046596,0.009697,0.988223,1.000000,0.796296,0.987591,0.889205,LogReg
1,0.036651,0.009522,0.993576,0.970588,0.868421,0.993405,0.954270,LogReg
2,0.041023,0.009850,0.992505,1.000000,0.870370,0.992260,0.948590,LogReg
3,0.042709,0.009563,0.994647,1.000000,0.905660,0.994522,0.962543,LogReg
4,0.035293,0.009994,0.989293,1.000000,0.843750,0.988870,0.931250,LogReg
5,0.117363,0.038593,0.987152,0.977273,0.796296,0.986533,0.891793,SVM
6,0.120370,0.034305,0.991435,0.916667,0.868421,0.991324,0.963728,SVM
7,0.122291,0.035726,0.994647,1.000000,0.907407,0.994524,0.956839,SVM
8,0.121519,0.037183,0.993576,0.979592,0.905660,0.993457,0.956525,SVM
9,0.126223,0.035290,0.989293,1.000000,0.843750,0.988870,0.913847,SVM


In [ ]:
def run_kernel_All_set(X_train_scaled: pd.DataFrame , y_train: pd.DataFrame, X_test_scaled: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame: 
    name=[]
    models = [
          ('Linear', SVC(kernel='linear')), 
          ('RBF', SVC(kernel='rbf')),
            ]
    target_names = ['No Re-buy', 'Re-buy']
    for name, model in models:
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(All_X_scaled)
        print(name)
        print(classification_report(All_y, y_pred, target_names=target_names))
       
      

In [ ]:
run_kernel_predict(X_train,y_train,X_test,y_test)